In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [45]:
joke=pd.read_csv('jokes.csv')
joke.set_index('joke_id',inplace=True)
joke.head(5)

,joke_text
joke_id,
1,Q. What's O. J. Simpson's web address? A. Slas...
2,How many feminists does it take to screw in a ...
3,Q. Did you hear about the dyslexic devil worsh...
4,They asked the Japanese visitor if they have e...
5,Q: What did the blind person say when given so...


In [47]:
t=pd.read_csv('train.csv')
t.set_index('joke_id',inplace=True)
t.head(1)

,id,user_id,Rating
joke_id,,,
110,31030_110,31030,2.75


In [48]:
t['joke']=joke['joke_text']
t['len']=joke['joke_text'].apply(len)

In [49]:
t.head(2)

,id,user_id,Rating,joke,len
joke_id,,,,,
110,31030_110,31030,2.750,"Judy was having trouble with her computer, so ...",562
109,16144_109,16144,5.094,One day the first grade teacher was reading th...,541


## Since jokes ratings are varying b/w positive and negative no., So we are going to consider jokes that have a rating above 0 as 1(positive) and rest as 0(negative).

In [50]:
def rate(x):
    if x>=0:
        return 1
    else:
        return 0
p=t.Rating.map(rate)
t.Rating=p
t.head()

,id,user_id,Rating,joke,len
joke_id,,,,,
110,31030_110,31030,1,"Judy was having trouble with her computer, so ...",562
109,16144_109,16144,1,One day the first grade teacher was reading th...,541
6,23098_6,23098,0,Q. What is orange and sounds like a parrot? A....,56
86,14273_86,14273,1,Two attorneys went into a diner and ordered tw...,329
134,18419_134,18419,1,A man is driving in the country one evening wh...,759


In [51]:
t=t.sort_index(kind='quicksort',axis=0)
t.head()

,id,user_id,Rating,joke,len
joke_id,,,,,
1,74_1,74,0,Q. What's O. J. Simpson's web address? A. Slas...,88
1,490_1,490,1,Q. What's O. J. Simpson's web address? A. Slas...,88
1,315_1,315,0,Q. What's O. J. Simpson's web address? A. Slas...,88
1,337_1,337,1,Q. What's O. J. Simpson's web address? A. Slas...,88
1,457_1,457,1,Q. What's O. J. Simpson's web address? A. Slas...,88


# Data Cleaning

In [8]:
t=t.drop_duplicates(subset={'joke','len'},keep='first')
t.head()

,id,user_id,Rating,joke,len
joke_id,,,,,
1,74_1,74,0,Q. What's O. J. Simpson's web address? A. Slas...,88
2,31674_2,31674,1,How many feminists does it take to screw in a ...,75
3,37340_3,37340,1,Q. Did you hear about the dyslexic devil worsh...,81
4,39978_4,39978,1,They asked the Japanese visitor if they have e...,99
5,5568_5,5568,0,Q: What did the blind person say when given so...,84


In [9]:
import re
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
stop = set(stopwords.words('english')) #set of stopwords
sno = SnowballStemmer('english') #initialising the snowball stemmer

def cleanpunc(sentence): #function to clean the word of any punctuation or special characters
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|:)|(|\|/]',r' ',cleaned)
    cleaned=cleaned.lower()
    return  cleaned
def clean(l):
    #for sent in s:
    filtered_sentence=[]
    #print(s)
    for cleaned_words in cleanpunc(l).split():
            if(len(cleaned_words)>2):    
                if(cleaned_words.lower() not in stop):
                    s=(sno.stem(cleaned_words.lower())).encode('utf8')
                    filtered_sentence.append(s)
        
    str1 = b" ".join(filtered_sentence) 
    #print(str1)
    return str1

In [12]:
final_string=[]
for i in t.joke:
    final_string.append(clean(i))
#final_string

In [13]:
t['cjoke']=final_string
t=t.drop('joke',axis=1)
t['len']=t['cjoke'].apply(len)
t.head()

,id,user_id,Rating,len,cjoke
joke_id,,,,,
1,74_1,74,0,64,b'what simpson web address slash slash backsla...
2,31674_2,31674,1,46,b'mani feminist take screw light bulb that funni'
3,37340_3,37340,1,41,b'hear dyslex devil worship sold soul santa'
4,39978_4,39978,1,51,b'ask japanes visitor elect countri everi morn...
5,5568_5,5568,0,40,b'blind person say given matzah hell wrote'


In [55]:
t.to_csv('finaljoke')


# This is our final data after cleaning and rearrangement

In [14]:
f=pd.read_csv('finaljoke',index_col='joke_id')
f.head()

,id,user_id,Rating,len,cjoke
joke_id,,,,,
1,74_1,74,0,64,b'what simpson web address slash slash backsla...
2,31674_2,31674,1,46,b'mani feminist take screw light bulb that funni'
3,37340_3,37340,1,41,b'hear dyslex devil worship sold soul santa'
4,39978_4,39978,1,51,b'ask japanes visitor elect countri everi morn...
5,5568_5,5568,0,40,b'blind person say given matzah hell wrote'


# Splitting the data into train and test set

In [37]:
from sklearn.model_selection import train_test_split
X=f.cjoke
y=f.Rating
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3,random_state=101)

# We are going to use bag of words technique on cjoke text 

In [38]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X_train=cv.fit_transform(X_train).toarray()
print(type(X_train))

<class 'numpy.ndarray'>


# We are using Naive Bayes classifier

In [39]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(X_train,y_train)
X_test=cv.transform(X_test).toarray()
predictions = nb.predict(X_test)

from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
print(accuracy_score(y_test,predictions))
print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))

0.5714285714285714
[[ 2 12]
 [ 6 22]]
             precision    recall  f1-score   support

          0       0.25      0.14      0.18        14
          1       0.65      0.79      0.71        28

avg / total       0.51      0.57      0.53        42



### since accuracy is low around 57% so we are going to use bagging technique i.e using multiple classifier to classify and then take the majority vote to classify if the joke is funny or not.

### we are going to use naive bayes,svm and linear regression

In [41]:
from sklearn import decomposition, ensemble
from sklearn import metrics,linear_model,svm
X=f.cjoke
y=f.Rating
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3,random_state=101)
X_train=cv.fit_transform(X_train).toarray()
X_test=cv.transform(X_test).toarray()
# Naive Bayes
nb = MultinomialNB()
nb.fit(X_train,y_train)
predictions = nb.predict(X_test)
accuracy=metrics.accuracy_score(predictions, y_test)
print("NB, Count Vectors: ", accuracy)
print(confusion_matrix(predictions, y_test))
# Linear Classifier 
lr=linear_model.LogisticRegression()
lr.fit(X_train,y_train)
predictions = lr.predict(X_test)
accuracy=metrics.accuracy_score(predictions, y_test)
print ("LR, Count Vectors: ", accuracy)
print(confusion_matrix(predictions, y_test))
# SVM 
SV=svm.SVC()
SV.fit(X_train,y_train)
predictions = SV.predict(X_test)
accuracy=metrics.accuracy_score(predictions, y_test)
print ("SVM, N-Gram Vectors: ", accuracy)
print(confusion_matrix(predictions, y_test))

NB, Count Vectors:  0.5714285714285714
[[ 2  6]
 [12 22]]
LR, Count Vectors:  0.6666666666666666
[[ 8  8]
 [ 6 20]]
SVM, N-Gram Vectors:  0.6666666666666666
[[ 0  0]
 [14 28]]


## input the joke you want to classify in j

In [52]:
j="How many feminists does it take to screw in a light bulb? That's not funny."
j=clean(j)
l=[]
l.append(j)
s=cv.transform(l).toarray()
a=int(SV.predict(s))
b=int(lr.predict(s))
c=int(nb.predict(s))
if(a+b+c>=2):
    print("funny")
else:
    print("not funny")

funny
